In [1]:
import os
import cv2
import mediapipe as mp
import numpy as np
from keras.models import load_model

In [2]:
actions = ['love', 'come', 'heart', 'hello', 'deny']
seq_length = 30

In [3]:
model = load_model(os.path.join('models', 'model.h5'))

In [4]:
# MediaPipe hands model
mp_hands= mp.solutions.hands
mp_drawing= mp.solutions.drawing_utils
hands= mp_hands.Hands(
    max_num_hands=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)

In [5]:
cap = cv2.VideoCapture(0)

In [6]:
# ## for recording the video clip
# os.makedirs('result', exist_ok=True)
# w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
# h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
# fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
# out = cv2.VideoWriter('result/input.mp4', fourcc, cap.get(cv2.CAP_PROP_FPS), (w, h))
# out2 = cv2.VideoWriter('result/output.mp4', fourcc, cap.get(cv2.CAP_PROP_FPS), (w, h))

In [7]:
seq = []
action_seq = []

In [8]:
while cap.isOpened():
    ret, img= cap.read()
    img0= img.copy()

    img= cv2.flip(img, 1)  # flipping the screen
    img= cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  ## BGR -> RGB

    result= hands.process(img)
    img= cv2.cvtColor(img, cv2.COLOR_RGB2BGR)  ## RGB -> BGR

    if result.multi_hand_landmarks is not None:
        for res in result.multi_hand_landmarks:
            joint= np.zeros((21, 4))
            for j, lm in enumerate(res.landmark):
                joint[j] = [lm.x, lm.y, lm.z, lm.visibility]

            # Compute angles between joints
            v1= joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19], :3] # Parent joint
            v2= joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20], :3] # Child joint
            v= v2 - v1 # [20, 3]
            # Normalize v
            v= v / np.linalg.norm(v, axis=1)[:, np.newaxis]  ## np.newaxis= expanding matrix dimensionality(in this case, add column)

            # Get angle using arcos of dot product
            angle= np.arccos(np.einsum('nt,nt->n',
                v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:], 
                v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:])) # [15,]
            angle= np.degrees(angle) # Convert radian to degree
            d= np.concatenate([joint.flatten(), angle])
            seq.append(d)

            mp_drawing.draw_landmarks(img, res, mp_hands.HAND_CONNECTIONS)

            if len(seq) < seq_length:
                continue
            input_data= np.expand_dims(np.array(seq[-seq_length:], dtype=np.float32), axis=0)
            y_pred= model.predict(input_data).squeeze()
            i_pred= int(np.argmax(y_pred))
            conf= y_pred[i_pred]

            if conf < 0.9:
                continue

            action= actions[i_pred]
            action_seq.append(action)
            if len(action_seq) < 3:
                continue

            this_action= '?'
            if action_seq[-1] == action_seq[-2] == action_seq[-3]:
                this_action= action

            cv2.putText(img, f'{this_action.upper()}', org=(int(res.landmark[0].x * img.shape[1]), \
                int(res.landmark[0].y * img.shape[0] + 20)), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 255, 255), thickness=2)

#     out.write(img0)
#     out2.write(img)
    cv2.imshow('img', img)
    if cv2.waitKey(1) == ord('q'):
        break

1/1 [==============================] - 0s 27ms/step


1/1 [==============================] - 0s 24ms/step


1/1 [==============================] - 0s 49ms/step


1/1 [==============================] - 0s 27ms/step


1/1 [==============================] - 0s 26ms/step


1/1 [==============================] - 0s 27ms/step


1/1 [==============================] - 0s 34ms/step


1/1 [==============================] - 0s 28ms/step
